In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nx = 200
nghost = 1
nxt = nx+2*nghost
xl, xr = 0., 200.
dx = (xr - xl) / nx
nu = 0.8
c = 1
epsiron = 0.1
L = 50

#'lax-wendroff', 'upwind'
scheme = 'upwind'

#'c-upwind'
out_dir = 'c-upwind'

dt = nu * dx / c
t = 0.
t_end = 200
nstep = 0
n_output = 20

In [3]:
#Node points
xn = np.linspace(xl-nghost*dx, xr+nghost*dx, nxt+1)

#Cell-centered points
xc = np.linspace(xl-0.5*nghost*dx, xr+0.5*nghost*dx, nxt)

In [4]:
#Initial conditions
k = 2 * np.pi / L
u = 1 + np.where(xc > L, 0, epsiron * np.sin(k * xc))

In [5]:
while t < t_end:
    u[0] = u[1]
    u[nxt-1] = u[nxt-2]
    
    u_right = u[nghost:]
    u_left = u[:-nghost]
    u_inner = u[nghost:-nghost]
    
    fc_left = u_left * u_left / 2
    fc_right = u_right * u_right / 2
    
    if scheme == 'upwind':
        fn = np.where(u_right + u_left > 0, fc_left, fc_right)
        fn_right = fn[1:]
        fn_left = fn[:-1]
    
    elif scheme == 'lax-wendroff':
        pass
        
    else:
        print('Error: Unknown scheme. Exsiting.')
        exit()
        
    u_new_inner  = u_inner - nu * (fn_right - fn_left)

    u_inner = u_new_inner.copy()
    u = np.concatenate(([0], u_inner, [0]))

    t = t + dt
    nstep = nstep + 1
    
    if nstep % n_output == 0:
        u_inner.tofile(f'{out_dir}/{scheme}-{int(nstep/n_output):0>4d}.dat')
        